In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('profiles.csv')

#print(df.describe())
print(df.info())
#df.head()

#EDA

#age column
#plt.hist(df['age'])
#plt.show() #based on the histogram, let's consider that everything above 80 years in wrong and drop those rows
df = df[df['age'] < 80]

#body type
# 3 groups with ~20% of the answers each. All the others have less than 10%. Will keep those 3 and aggregate the others

map_body_type = {
    'average': 'average',
    'fit': 'fit', 
    'athletic': 'athletic',  
    'thin': 'other', 
    'curvy': 'other',
    'a little extra': 'other',  
    'skinny': 'other', 
    'full figured': 'other',  
    'overweight': 'other',  
    'jacked': 'other', 
    'used up': 'unknown',  
    'rather not say': 'unknown'
}

df['body_type'] = df['body_type'].map(map_body_type).fillna('unknown')

#diet
map_diet = {
    # Group 1: Anything
    'mostly anything': 'anything',
    'anything': 'anything',
    'strictly anything': 'anything',
    
    # Group 2: Vegetarian/Vegan
    'vegetarian': 'vegetarian',
    'mostly vegetarian': 'vegetarian',
    'strictly vegetarian': 'vegetarian',
    'vegan': 'vegetarian',
    'mostly vegan': 'vegetarian',
    'strictly vegan': 'vegetarian',
    
    # Group 3: Other (includes religious-based + remaining 'other')
    'other': 'other',
    'mostly other': 'other',
    'strictly other': 'other',
    'kosher': 'other',
    'mostly kosher': 'other',
    'strictly kosher': 'other',
    'halal': 'other',
    'mostly halal': 'other',
    'strictly halal': 'other'
}

df['diet'] = df['diet'].map(map_diet).fillna('unknown')
# ~40% of unknowns, ~50% of anything. Might consider dropping this one

#drinks

map_drinks = {
    'socially': 'socially',
    'rarely': 'light',
    'not at all': 'light',
    'often': 'heavy',
    'very often': 'heavy',
    'desperately': 'heavy'
}
df['drinks'] = df['drinks'].map(map_drinks).fillna('unknown')

#socially category dominates with 70%

#drugs
map_drugs = {
    'never': 'no',
    'sometimes': 'yes',
    'often': 'yes',
    'unknown': 'unknown'
}

df['drugs'] = df['drugs'].map(map_drugs).fillna('unknown')

#education

#will divide into education level and is student

education_level_conditions = [
    df['education'].str.contains('college', case = False, na=False),
    df['education'].str.contains('masters', case = False, na=False),
    df['education'].str.contains('high school|ph.d|law school|med school', case = False, na=False) #these represent a low percentage <5%, so will group them
]

education_level_choices = ['college', 'masters', 'other']

df['education_level'] = np.select(education_level_conditions, education_level_choices, default = 'unknown')

#print(df['education_level'].value_counts(normalize = True))

is_student_conditions = [
    df['education'].str.contains('working on', case = False, na=False),
    df['education'].str.contains('graduated from|dropped out', case = False, na=False),
    df['education'].str.contains('space camp', case = False, na=False) #will consider all these unknown    
]

is_student_choices = ['yes', 'no', 'unknown']

df['is_student'] = np.select(is_student_conditions, is_student_choices, default = 'unknown')

df = df.drop('education', axis = 1)

ethnicity_conditions = [
    df['ethnicity'] == 'white',
    df['ethnicity'] == 'asian',
    df['ethnicity'].isna()
]

ethnicity_choices = ['white', 'asian', 'unknown']

df['ethnicity'] = np.select(ethnicity_conditions, ethnicity_choices, default = 'other')

#height
#plt.hist(df['height'])
#plt.show()
df = df[(df['height'] >= 50) & (df['height'] <= 90)] #this is a reasonable interval for height

#income
# 80% of unknowns, not a great column to predict other columns, but might be fun to remove the unknowns and predict this one

#plt.hist(df_income['income'])
#plt.show()

def classify_income(i):
    if i <= 0:
        return 'unknown'
    elif i <= 20000:
        return 'low'
    elif i <= 50000:
        return 'medium'
    else: 
        return 'high'

df['income'] = df['income'].apply(classify_income)

#print(df['income'].value_counts(normalize = True))

#job

job_map = {
    # STEM/Technical
    'science / tech / engineering': 'stem',
    'computer / hardware / software': 'stem',

    # Creative/Artistic
    'artistic / musical / writer': 'creative',
    'entertainment / media': 'creative',

    # Business/Professional
    'sales / marketing / biz dev': 'business',
    'executive / management': 'business',
    'banking / financial / real estate': 'business',
    'law / legal services': 'business',

    # Education/Health
    'education / academia': 'education_health',
    'medicine / health': 'education_health',

    # Manual Labor / Skilled Trades - Not enough percentage to create its own category
    'construction / craftsmanship': 'other',
    'transportation': 'other',
    'clerical / administrative': 'other',

    # Public/Government - Not enough percentage to create its own category
    'political / government': 'other',
    'military': 'other',

    # Other groups
    'student': 'student',
    'unemployed': 'other',
    'retired': 'other',
    'rather not say': 'unknown',
    'hospitality / travel': 'other',
    'other': 'other'
}

df['job'] = df['job'].map(job_map).fillna('unknown')

#last online

df['last_online'] = pd.to_datetime(df['last_online'], format='%Y-%m-%d-%H-%M')
most_recent_date = df['last_online'].max()
df['last_online'] = (most_recent_date - df['last_online']).dt.days

#binary distribution into active or not active. Could also make a semi active category (<=1, <=3, >3)
def categorize_last_online(v):
    if v <= 3:
        return 'active'
    else:
        return 'not active'

df['last_online'] = df['last_online'].apply(categorize_last_online)

#location

def get_city(location):
    city = location.split(',')[0]
    if city == 'san francisco': #50% of the states are san francisco, so will do this and 'other'
        return city
    else:
        return 'other'

df['city'] = df['location'].apply(get_city)

#offspring
df['offspring'] = df['offspring'].fillna('unknown')

#will drop the column because there are 60% of unknowns and spliting between has and wants kids made it worse.

df.drop('offspring', axis = 1)

#orientation

map_orientation = {
    'straight': 'straight',
    'gay': 'queer',
    'bisexual': 'queer'
}

df['orientation'] = df['orientation'].map(map_orientation).fillna('unknown')

#pets
df['pets'] = df['pets'].fillna('unknown')

def likes_dogs(pets):
    if isinstance(pets, str):
        pets = pets.lower().strip()
        if 'likes dogs' in pets or 'has dogs' in pets:
            return 'yes'
    return 'no'

def likes_cats(pets):
    if isinstance(pets, str):
        pets = pets.lower().strip()
        if 'likes cats' in pets or 'has cats' in pets:
            return 'yes'
    return 'no'
        

df['likes_dogs'] = df['pets'].apply(likes_dogs)
df['likes_cats'] = df['pets'].apply(likes_cats)

#religion
df['religion'] = df['religion'].fillna('unknown')

def get_religion(r):
    try:
        religion = r.split(' ')[0]
    except:
        print(r)
        return
    if religion == 'buddhism' or religion == 'hinduism' or religion == 'islam':
        return 'other'
    else:
        return religion

religion_importance_conditions = [
    df['religion'].str.contains('not too serious|laughing', case=False, na=False),
    df['religion'].str.contains('somewhat serious|very serious', case=False, na=False),
    df['religion'].str.contains('unknown', case=False, na=False)
]

religion_importance_options = ['no', 'yes', 'unknown']

df['religion_importance'] = np.select(religion_importance_conditions, religion_importance_options, default = 'unknown')
df['religion'] = df['religion'].apply(get_religion)   

#sex - good to go
#print(df['sex'].value_counts(normalize=True, dropna=False))

#sign
df['sign'] = df['sign'].fillna('unknown')
df['sign'] = df['sign'].str.replace('&rsquo;', '\'', regex=False)

def get_sign(sign):
    if isinstance(sign, str):
        return sign.split(' ')[0]
    else:
        return 'unknown'

sign_importance_conditions = [
    df['sign'].str.contains('fun', regex = False),
    df['sign'].str.contains('but it doesn\'t matter', regex = False),
    df['sign'].str.contains('and it matters a lot', regex = False)
]

sign_importance_options = ['fun', 'not important', 'important']

df['sign_importance'] = np.select(sign_importance_conditions, sign_importance_options, default = 'unknown')
df['sign'] = df['sign'].apply(get_sign)

#print(df['sign'].value_counts(normalize=True, dropna=False)) # 12 categories + unknown -> not sure if it will have predictive power. Might drop it
#print(df['sign_importance'].value_counts(normalize=True, dropna=False))  # only 1% considers important, might drop it

df.drop(['sign', 'sign_importance'], axis = 1)

#smokes
df['smokes'] = df['smokes'].fillna('unknown')

smokes_map = {
    'no': 'no',
    'unknown': 'unknown',
    'sometimes': 'yes',
    'when drinking': 'yes',
    'trying to quit': 'yes',
    'yes': 'yes'
}

df['smokes'] = df['smokes'].map(smokes_map)

#print(df['smokes'].value_counts(normalize=True, dropna=False))

#speaks - has a list of comma separated values with proficiency between brackets
# will get the top 5 languages and save the proficiency level for each + add a column for languages count
df['speaks'] = df['speaks'].fillna('unknown')

def get_languages_count(l):
    count_commas = l.count(',')
    return (count_commas + 1)

df['language_count'] = df['speaks'].apply(get_languages_count)
#print(df['language_count'].value_counts(normalize=True, dropna=False))

#TO DO -> get the 5 top languages and create columns with the level

language_counts = {}
def get_top_languages(lang):
    languages = lang.split(',')
    for language in languages:
        l = language.strip().split(' ')[0]
        if l in language_counts:
            language_counts[l] += 1
        else:
            language_counts[l] = 1

df['speaks'].apply(get_top_languages)
top_5_languages = sorted(language_counts, key=language_counts.get, reverse=True)[:5]
print(top_5_languages)

for lang in top_5_languages:
    df[f'{lang}_level'] = 'none'

for idx, row in df['speaks'].items():
    languages = row.split(',')
    for language in languages:
        record = language.strip().split(' ')
        l = record[0].strip().lower()
        if len(record) == 1:
            level = 'okay'
        elif len(record) >= 2:
            level = record[1].strip('()').lower()
        if l in top_5_languages:
            df.at[idx, f'{l}_level'] = level        

#status
df['status'] = df['status'].fillna('unknown')
#print(df['status'].value_counts(normalize=True, dropna=False))

status_map = {
    'single': 'available',
    'seeing someone': 'unavailable',
    'available': 'available',
    'married': 'unavailable',
    'unknown': 'unknown'
}

df['status'] = df['status'].map(status_map)

#print(df['status'].value_counts(normalize=True, dropna=False))

df = df.drop(['offspring', 'location', 'pets', 'speaks', 'is_student'], axis=1)
not_essay_cols = [col for col in df.columns if 'essay' not in col]
for col in not_essay_cols:
    if df[col].dtype in ['object', 'category']:
        print(df[col].value_counts(normalize=True))




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          59946 non-null  int64  
 1   body_type    54650 non-null  object 
 2   diet         35551 non-null  object 
 3   drinks       56961 non-null  object 
 4   drugs        45866 non-null  object 
 5   education    53318 non-null  object 
 6   essay0       54458 non-null  object 
 7   essay1       52374 non-null  object 
 8   essay2       50308 non-null  object 
 9   essay3       48470 non-null  object 
 10  essay4       49409 non-null  object 
 11  essay5       49096 non-null  object 
 12  essay6       46175 non-null  object 
 13  essay7       47495 non-null  object 
 14  essay8       40721 non-null  object 
 15  essay9       47343 non-null  object 
 16  ethnicity    54266 non-null  object 
 17  height       59943 non-null  float64
 18  income       59946 non-null  int64  
 19  job 

In [11]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold, RFE, SelectKBest, chi2
import seaborn as sns
from xgboost import XGBClassifier
from collections import Counter
#from imblearn.over_sampling import SMOTE

def downsample(df, col):
    min_count = df[col].value_counts().min()
    balanced_df = pd.concat([
        df[df[col] == category].sample(n=min_count, random_state=1)
        for category in df[col].unique()
    ])
    return balanced_df

data = df[not_essay_cols].copy()
col_to_predict = 'income'
print(data[col_to_predict].value_counts())

#remove rows where the target variable is unknown
data = data[data[col_to_predict] != 'unknown']
data = data[data[col_to_predict].notna()]

data = downsample(data, col_to_predict)

#create vars with categorical columns and numerical columns
cat_cols = data.select_dtypes(include=['object','category']).columns.tolist()
cat_cols = [col for col in cat_cols if col != col_to_predict]
num_cols = data.select_dtypes(include=['float64', 'int64']).columns.tolist()
num_cols = [col for col in num_cols if col != col_to_predict]

df_dummies = pd.get_dummies(data[cat_cols], drop_first = True)

x = pd.concat([data[num_cols], df_dummies], axis = 1)
y = data[col_to_predict]

print(data[col_to_predict].value_counts()) #check if the classes are balanced

le = LabelEncoder()
y = le.fit_transform(y)
print(le.classes_)

# ------------ Chi2 test ------------ #
# Chi2 requires all values to be non-negative
X_chi = x.copy()
X_chi[X_chi < 0] = 0  # Only necessary if you have negative values

selector = SelectKBest(score_func=chi2, k=10)  # Keep top 10 features
X_chi_selected = selector.fit_transform(X_chi, y)
selected_features = X_chi.columns[selector.get_support()]
print(f"Selected features from Chi2: {list(selected_features)}")

x = x[selected_features]

cat_cols = x.select_dtypes(include=['object','category']).columns.tolist()
num_cols = x.select_dtypes(include=['float64', 'int64']).columns.tolist()

# ------------ End of Chi2 test ------------ #

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=24, stratify=y)

scaler = StandardScaler()
if len(num_cols) > 0:
    x_train[num_cols] = scaler.fit_transform(x_train[num_cols])
    x_test[num_cols] = scaler.transform(x_test[num_cols]) #never use fit transform on test data, because it will learn from train data

print(pd.Series(y_train).value_counts(normalize=True))

# ----- XGB Classifier ----- #

model = XGBClassifier(eval_metric='logloss')

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

gs = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='f1_macro', verbose=1, n_jobs=-1)    

gs.fit(x_train, y_train)
y_pred = gs.predict(x_test)
print('Classification report for XGB Classifier:')
print(classification_report(y_test, y_pred))
print("Best Parameters:", gs.best_params_)

# --------- Logistic Regression ------------- #

model = LogisticRegression(max_iter = 1000)

param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
    'solver': ['liblinear', 'saga'],  # Solvers that support l1 and elasticnet
    'class_weight': [None, 'balanced'],
    'l1_ratio': [0, 0.5, 1]  # Only used with elasticnet
}
gs = GridSearchCV(model, param_grid, cv=5, scoring='f1_weighted', verbose=1, n_jobs=-1)

gs.fit(x_train, y_train)
y_pred_model = gs.predict(x_test)
model_score = classification_report(y_test, y_pred_model)
print('Classification report for LR:')
print(model_score)
print("Best Parameters:", gs.best_params_)

#TRY K NEAREST NEIGHBORS AND DECISION TREE

#USE NAIVE BAYS CLASSIFIER TO PREDICT SOMETHING BASED ON THE ESSAYS


income
unknown    48404
high        5509
medium      3026
low         2948
Name: count, dtype: int64
income
high      2948
low       2948
medium    2948
Name: count, dtype: int64
['high' 'low' 'medium']
Selected features from Chi2: ['age', 'body_type_other', 'drugs_yes', 'job_other', 'job_stem', 'job_student', 'sign_unknown', 'smokes_yes', 'education_level_masters', 'city_san francisco']
0    0.333428
1    0.333286
2    0.333286
Name: proportion, dtype: float64
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Classification report for XGB Classifier:
              precision    recall  f1-score   support

           0       0.61      0.66      0.63       589
           1       0.70      0.62      0.65       590
           2       0.46      0.47      0.46       590

    accuracy                           0.58      1769
   macro avg       0.59      0.58      0.58      1769
weighted avg       0.59      0.58      0.58      1769

Best Parameters: {'colsample_bytree': 0.8, 'learn

C:\Users\pedro\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
450 fits failed out of a total of 1200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\pedro\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\pedro\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pedro\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1194, in fit
    solver

Classification report for LR:
              precision    recall  f1-score   support

           0       0.61      0.65      0.63       589
           1       0.66      0.68      0.67       590
           2       0.47      0.42      0.44       590

    accuracy                           0.58      1769
   macro avg       0.58      0.58      0.58      1769
weighted avg       0.58      0.58      0.58      1769

Best Parameters: {'C': 10, 'class_weight': 'balanced', 'l1_ratio': 0.5, 'penalty': 'elasticnet', 'solver': 'saga'}
